In [ ]:
!pip install -I xpander-sdk openai python-dotenv

In [7]:
from xpander_sdk import XpanderClient
from openai import OpenAI
from dotenv import load_dotenv
import os

## Load environment variables
load_dotenv()
friendli_client = OpenAI(
    base_url="https://api.friendli.ai/serverless/v1",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)

xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")


xpander_client = XpanderClient(api_key=xpanderAPIKey)
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

tools = xpander_agent.retrieve_all_graph_tools()
print(tools)


[{'type': 'function', 'function': {'name': 'SlackMessageSendingSendMessageToChannel', 'description': 'Sends a message to a channel. If the channel is missing, run listAllChannelsInTeam before anything else to get channel IDs. Use this method to post new messages, announcements, or updates to specific conversations or users. Common uses include team communication, automated notifications, and project updates. Supports rich content like attachments and blocks for enhanced messaging.', 'parameters': {'type': 'object', 'properties': {'bodyParams': {'type': 'object', 'properties': {'as_user': {'type': 'string', 'description': 'Pass true to post the message as the authenticated user instead of as a bot. Defaults to false. Use this to maintain the context of a user when posting. This option is only available for legacy Slack apps.'}, 'attachments': {'type': 'string', 'description': 'A JSON-based array of structured attachments, presented as a URL-encoded string. Use attachments to include ric

In [3]:
# verify the friendli client is working
completion = friendli_client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [6]:
memory = [{"role": "system", "content": "You are a helpful assistant. You are running in While True Loop, the response to your tools are available, once you have completed the user request add ##FINAL ANSWER## in your reply without further function calling"},
          {"role": "user", "content": "Send hi to C08628AHP9C channel"}]
print(memory[-1])
number_of_calls = 1
while True:
    tools = xpander_agent.retrieve_all_graph_tools()
    llm_response = friendli_client.chat.completions.create(
        model="meta-llama-3.1-70b-instruct",
        messages=memory,
        tools=tools
        )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    model_response = llm_response.choices[0].message
    print(model_response)
    memory.append({"role": "assistant", "content": model_response.content})
    if model_response.tool_calls:
        print("model decided to invoke tools")
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump())
        for tool in tools_to_run: 
            print(tool.name)
        tool_responses = xpander_agent.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            print("tool response for", tool_response.function_name, tool_response.status_code)
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
    if (model_response.content):
        if "##FINAL ANSWER##" in model_response.content:
            break
    number_of_calls += 1
print(model_response.content)   

{'role': 'user', 'content': 'Send hi to C08628AHP9C channel'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WcYhGOR0FfuSpHRnTfBBbHGO', function=Function(arguments='{"bodyParams": {"channel": "C08628AHP9C", "text": "hi", "unfurl_links": false, "unfurl_media": false, "username": "hi-bot"}, "queryParams": {}, "pathParams": {}}', name='SlackMessageSendingSendMessageToChannel'), type='function')])
model decided to invoke tools
SlackMessageSendingSendMessageToChannel
tool response for SlackMessageSendingSendMessageToChannel 200
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ensF5YRlPmjnnEKPjld7nx9r', function=Function(arguments='{"bodyParams": {"channel": "C08628AHP9C", "text": "hi", "username": "My Slack Bot"}, "queryParams": {}, "pathParams": {}}', name='SlackMessageSendingSendMessage